# 검색증강생성(RAG)을 활용한 챗봇 구현

#### 앞 부분은 이전 실습 과정에서 했던 내용과 동일합니다


In [1]:
import boto3
%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [2]:
%load_ext autoreload
%autoreload 2

### Bedrock 설정 (이전 실습과 동일)

In [3]:
from botocore.config import Config
from langchain_community.chat_models import BedrockChat
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [4]:
region_name = 'us-east-1'
retry_config = Config(
        region_name=region_name,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
boto3_bedrock = boto3.client("bedrock-runtime", region_name=region_name, config=retry_config)

llmchat = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=boto3_bedrock,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"]
    }
)
llmchat

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7f617e0d4970>, model_id='anthropic.claude-3-sonnet-20240229-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f617e07b550>])

In [11]:
from langchain.embeddings import BedrockEmbeddings

In [12]:
llmemb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-g1-text-02"
)
dimension = 1536
print("Bedrock Embeddings Model Loaded")

Bedrock Embeddings Model Loaded


### OpenSearch 설정 (이전 실습과 동일)

In [5]:
from utils.rag import qa_chain
from utils.rag import prompt_repo, show_context_used
from utils.rag import retriever_utils, OpenSearchHybridSearchRetriever
from opensearchpy import OpenSearch, RequestsHttpConnection

In [8]:
http_auth = (opensearch_user_id, opensearch_user_password)
os_client = OpenSearch(
        hosts=[
            {'host': opensearch_domain_endpoint.replace("https://", ""),
             'port': 443
            }
        ],
        http_auth=http_auth, 
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection
    )

### 검색어에 대한 정보획득을 위한 OpenSearch Retriever 클래스 생성

정보획득 구현방식에 여러 파라미터를 적용하기 위한 클래스로, 세부 동작방식은 `utils/rag.py`에 정의되어 있습니다

In [17]:
index_name = "sample_pdf"
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    llm_text=llmchat, 
    llm_emb=llmemb,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for search
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[1.0, 0.0], # [for semantic, for lexical], Semantic, Lexical search 결과에 대한 최종 반영 비율 정의
    reranker=False, 
    parent_document = False, # enable parent document
    
    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=False,
)

In [23]:
system_prompt = prompt_repo.get_system_prompt()
# 기본 프롬프트 템플릿을 불러와서 활용
print(system_prompt)


                        You are a master answer bot designed to answer user's questions.
                        I'm going to give you contexts which consist of texts, tables and images.
                        Read the contexts carefully, because I'm going to ask you a question about it.
                        


In [19]:
qa = qa_chain(
    llm_text=llmchat,
    retriever=opensearch_hybrid_retriever,
    system_prompt=system_prompt,
    return_context=True,
    verbose=False
)


In [26]:
query = "특별비용담보 특별약관에서 회사가 보상하는 비용의 범위는?"

response, contexts = qa.invoke(
    query = query
)

print("\n\n\n==============아래는 위 답변에 사용된 컨텍스트입니다==============\n")
show_context_used(contexts)

제2조(비용의 범위)에서 회사가 보상하는 비용의 범위는 다음과 같습니다.

1. 수색구조비용: 조난당한 피보험자를 수색, 구조 또는 이송하는 활동에 필요한 비용
2. 항공운임등 교통비: 피보험자의 수색, 간호 또는 사고처리를 위한 구원자의 현지 왕복 교통비 (2명분 한도)
3. 숙박비: 구원자의 현지 숙박비 (2명분, 1명당 14일분 한도) 
4. 이송비용: 피보험자 유해 또는 치료중인 피보험자 이송 비용
5. 제잡비: 구원자의 출입국 절차 비용, 현지 교통비, 통신비, 유해처리비 등


==============아래는 위 답변에 사용된 컨텍스트입니다==============


-----------------------------------------------
1. Chunk: 539 Characters
-----------------------------------------------
10만원을 한도로 합니다. 제3조(보상하지 아니하는 손해) 회사는 보통약관 제7조(보상하지 아니하는 손해) 제1항 제1호 내지 제3호, 제7호 내지 제12호의 사유로 인하여 생긴
손해는 보상하여 드리지 아니합니다. 제4조(보험금의 지급) 회사는 제2조(비용의 범위)의 비용중 정당하다고 인정된 부분에 대해서 만 보상하여 드리며, 계약자, 피보험자 또는
보험수익자가 타인으로부터 손해배상을 받을 수 있는 경우에는 그 금액을 지급하지 아니합니다. 제5조(보험금의 분담) 제1조(보상하는 손해)의 비용에 대하여 보험금을 지급할 다수의
계약이 체결되어 있는 경우에는 각각의 계약에 대하여 다른 계약이 없는 것으로 하여 산출한 보상책 임액의 합계액이 그 비용을 초과했을 때 회사는 이 계약에 따른 보상책임액의 위의
합계액에 대한 비율에 따라 보험금을 지급하여 드립니다. 제6조(보상한도액) 회사가 이 특별약관에 관하여 지급할 보험금은 보험기간을 통하여 이 특별 약관의 보험가입금액을 한도로
합니다. 제7조(준용규정) 이 특별약관에 정하지 아니한 사항은 보통약관을 따릅니다.
metadata:
 